In [ ]:
## This file implements neural networks and logistic regression on p0017Spresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 99.19% (97.58% after improvements and regularization) for combination data
## and 97.9% for over-sampling data after improvements and regularization.
## For logistic regression, the accuracies are 97.58% for combination data and 95.1% for over-sampling data.
## For random forest, the accuracy is 99.19% for combination and 95.8% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.26% for combination data, and 97.28% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017Spresabs_quant.csv')
df.shape

(255, 146)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.214333
1      0.350500
2      0.517333
3      0.188833
4      0.191833
5      0.348667
6      0.182667
7      0.207000
8      0.405500
9      0.173500
10     0.184000
11     0.528667
12     0.211667
13     0.218667
14     0.167500
15     0.183333
16     0.406833
17     0.407333
18     0.321167
19     0.177167
20     0.271000
21     0.241833
22     0.178333
23     0.172500
24     0.205833
25     0.170000
26     0.224667
27     0.170500
28     0.163667
29     0.471000
         ...   
225    0.303700
226    0.310600
227    0.443300
228    0.388000
229    0.541200
230    0.378667
231    0.242167
232    0.261500
233    0.295833
234    0.193167
235    0.391333
236    0.160500
237    0.159333
238    0.172000
239    0.262000
240    0.244833
241    0.279833
242    0.200167
243    0.168333
244    0.548667
245    0.164167
246    0.202333
247    0.194167
248    0.188167
249    0.203333
250    0.444667
251    0.238000
252    0.167000
253    0.314333
254    0.252000
Name: pheno, Length: 255

In [5]:
df.head()

,id,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,...,group_2403,group_3458,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,pheno
0,107,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.214333
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0.350500
2,115,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.517333
3,120335,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.188833
4,120337,1,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0.191833


In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 146)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 145)

In [12]:
df_clean.head()

,TTTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTT,TTTTCCAGTAAT,TTTTAATACATAT,TTTTAAATATTATAA,TTTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTA,TTTATCTTTATGA,TTTAATTTAGTAAGT,TTTAAAAAGATGAATAATGTAAATGAAGTAAAGGTTATTATGAGAATTACAAAAGCTACATAAATTACTGTTAGTTTAAATTGAAATTTAAAAATGATAA,TTGAAGCATTAAGATTACTTATCATTTTTAAATTTCAATTTAAACTAACAGTAATTTATGTAGCTTTTGTAATTCTCATAATAACCTTTACTTCATTTAC,TTCCATCGAATCAC,...,group_2403,group_3458,group_3904,group_426,group_475,group_6375,group_7822,group_8071,group_8913,pheno
0,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 144) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 197), (1, 214)]


Using TensorFlow backend.


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [30]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [31]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 287 samples, validate on 124 samples
Epoch 1/1000
287/287 [==============================] - 0s 426us/step - loss: 0.7146 - accuracy: 0.5157 - val_loss: 0.6721 - val_accuracy: 0.5645
Epoch 2/1000
287/287 [==============================] - 0s 91us/step - loss: 0.6768 - accuracy: 0.5610 - val_loss: 0.6505 - val_accuracy: 0.6048
Epoch 3/1000
287/287 [==============================] - 0s 160us/step - loss: 0.6544 - accuracy: 0.5749 - val_loss: 0.6336 - val_accuracy: 0.6290
Epoch 4/1000
287/287 [==============================] - 0s 111us/step - loss: 0.6373 - accuracy: 0.5784 - val_loss: 0.6192 - val_accuracy: 0.6371
Epoch 5/1000
287/287 [==============================] - 0s 100us/step - loss: 0.6235 - accuracy: 0.6516 - val_loss: 0.6065 - val_accuracy: 0.6452
Epoch 6/1000
287/287 [==============================] - 0s 103us/step - loss: 0.6084 - accuracy: 0.6551 - val_loss: 0.5948 - val_accuracy: 0.6694
Epoch 7/1000
287/287 [==============================] - 0s 90us/step - loss: 0.

287/287 [==============================] - 0s 127us/step - loss: 0.3337 - accuracy: 0.8955 - val_loss: 0.3479 - val_accuracy: 0.8871
Epoch 57/1000
287/287 [==============================] - 0s 95us/step - loss: 0.3298 - accuracy: 0.8955 - val_loss: 0.3439 - val_accuracy: 0.8952
Epoch 58/1000
287/287 [==============================] - 0s 90us/step - loss: 0.3270 - accuracy: 0.8990 - val_loss: 0.3428 - val_accuracy: 0.8871
Epoch 59/1000
287/287 [==============================] - 0s 92us/step - loss: 0.3241 - accuracy: 0.8990 - val_loss: 0.3400 - val_accuracy: 0.8871
Epoch 60/1000
287/287 [==============================] - 0s 87us/step - loss: 0.3213 - accuracy: 0.9024 - val_loss: 0.3381 - val_accuracy: 0.8871
Epoch 61/1000
287/287 [==============================] - 0s 81us/step - loss: 0.3190 - accuracy: 0.9024 - val_loss: 0.3346 - val_accuracy: 0.8952
Epoch 62/1000
287/287 [==============================] - 0s 77us/step - loss: 0.3151 - accuracy: 0.9059 - val_loss: 0.3331 - val_accuracy

287/287 [==============================] - 0s 84us/step - loss: 0.2189 - accuracy: 0.9233 - val_loss: 0.2474 - val_accuracy: 0.9194
Epoch 112/1000
287/287 [==============================] - 0s 84us/step - loss: 0.2184 - accuracy: 0.9233 - val_loss: 0.2456 - val_accuracy: 0.9194
Epoch 113/1000
287/287 [==============================] - 0s 102us/step - loss: 0.2162 - accuracy: 0.9268 - val_loss: 0.2521 - val_accuracy: 0.9113
Epoch 114/1000
287/287 [==============================] - 0s 87us/step - loss: 0.2142 - accuracy: 0.9408 - val_loss: 0.2438 - val_accuracy: 0.9194
Epoch 115/1000
287/287 [==============================] - 0s 77us/step - loss: 0.2135 - accuracy: 0.9233 - val_loss: 0.2420 - val_accuracy: 0.9194
Epoch 116/1000
287/287 [==============================] - 0s 95us/step - loss: 0.2120 - accuracy: 0.9199 - val_loss: 0.2420 - val_accuracy: 0.9435
Epoch 117/1000
287/287 [==============================] - 0s 93us/step - loss: 0.2111 - accuracy: 0.9303 - val_loss: 0.2403 - val_ac

Epoch 167/1000
287/287 [==============================] - 0s 78us/step - loss: 0.1608 - accuracy: 0.9686 - val_loss: 0.1988 - val_accuracy: 0.9516
Epoch 168/1000
287/287 [==============================] - 0s 87us/step - loss: 0.1597 - accuracy: 0.9686 - val_loss: 0.1986 - val_accuracy: 0.9516
Epoch 169/1000
287/287 [==============================] - 0s 90us/step - loss: 0.1579 - accuracy: 0.9721 - val_loss: 0.1971 - val_accuracy: 0.9516
Epoch 170/1000
287/287 [==============================] - 0s 93us/step - loss: 0.1584 - accuracy: 0.9652 - val_loss: 0.1957 - val_accuracy: 0.9516
Epoch 171/1000
287/287 [==============================] - 0s 78us/step - loss: 0.1570 - accuracy: 0.9652 - val_loss: 0.1971 - val_accuracy: 0.9516
Epoch 172/1000
287/287 [==============================] - 0s 79us/step - loss: 0.1561 - accuracy: 0.9756 - val_loss: 0.1943 - val_accuracy: 0.9516
Epoch 173/1000
287/287 [==============================] - 0s 79us/step - loss: 0.1561 - accuracy: 0.9652 - val_loss: 0

Epoch 223/1000
287/287 [==============================] - 0s 82us/step - loss: 0.1252 - accuracy: 0.9756 - val_loss: 0.1725 - val_accuracy: 0.9758
Epoch 224/1000
287/287 [==============================] - 0s 85us/step - loss: 0.1247 - accuracy: 0.9791 - val_loss: 0.1700 - val_accuracy: 0.9758
Epoch 225/1000
287/287 [==============================] - 0s 77us/step - loss: 0.1259 - accuracy: 0.9756 - val_loss: 0.1700 - val_accuracy: 0.9758
Epoch 226/1000
287/287 [==============================] - 0s 81us/step - loss: 0.1254 - accuracy: 0.9721 - val_loss: 0.1720 - val_accuracy: 0.9758
Epoch 227/1000
287/287 [==============================] - 0s 76us/step - loss: 0.1237 - accuracy: 0.9791 - val_loss: 0.1692 - val_accuracy: 0.9758
Epoch 228/1000
287/287 [==============================] - 0s 103us/step - loss: 0.1231 - accuracy: 0.9686 - val_loss: 0.1745 - val_accuracy: 0.9758
Epoch 229/1000
287/287 [==============================] - 0s 80us/step - loss: 0.1243 - accuracy: 0.9791 - val_loss: 

...

Epoch 726/1000
287/287 [==============================] - 0s 79us/step - loss: 0.0384 - accuracy: 0.9895 - val_loss: 0.1328 - val_accuracy: 0.9839
Epoch 727/1000
287/287 [==============================] - 0s 88us/step - loss: 0.0379 - accuracy: 0.9895 - val_loss: 0.1336 - val_accuracy: 0.9839
Epoch 728/1000
287/287 [==============================] - 0s 83us/step - loss: 0.0381 - accuracy: 0.9895 - val_loss: 0.1328 - val_accuracy: 0.9839
Epoch 729/1000
287/287 [==============================] - 0s 79us/step - loss: 0.0384 - accuracy: 0.9895 - val_loss: 0.1328 - val_accuracy: 0.9839
Epoch 730/1000
287/287 [==============================] - 0s 75us/step - loss: 0.0385 - accuracy: 0.9895 - val_loss: 0.1327 - val_accuracy: 0.9839
Epoch 731/1000
287/287 [==============================] - 0s 84us/step - loss: 0.0379 - accuracy: 0.9895 - val_loss: 0.1331 - val_accuracy: 0.9839
Epoch 732/1000
287/287 [==============================] - 0s 83us/step - loss: 0.0377 - accuracy: 0.9895 - val_loss: 0

287/287 [==============================] - 0s 83us/step - loss: 0.0346 - accuracy: 1.0000 - val_loss: 0.1344 - val_accuracy: 0.9839
Epoch 782/1000
287/287 [==============================] - 0s 85us/step - loss: 0.0347 - accuracy: 0.9895 - val_loss: 0.1338 - val_accuracy: 0.9919
Epoch 783/1000
287/287 [==============================] - 0s 81us/step - loss: 0.0344 - accuracy: 1.0000 - val_loss: 0.1347 - val_accuracy: 0.9839
Epoch 784/1000
287/287 [==============================] - 0s 75us/step - loss: 0.0343 - accuracy: 0.9895 - val_loss: 0.1338 - val_accuracy: 0.9919
Epoch 785/1000
287/287 [==============================] - 0s 83us/step - loss: 0.0343 - accuracy: 0.9895 - val_loss: 0.1337 - val_accuracy: 0.9919
Epoch 786/1000
287/287 [==============================] - 0s 74us/step - loss: 0.0342 - accuracy: 0.9965 - val_loss: 0.1339 - val_accuracy: 0.9919
Epoch 787/1000
287/287 [==============================] - 0s 83us/step - loss: 0.0344 - accuracy: 0.9965 - val_loss: 0.1342 - val_acc

Epoch 837/1000
287/287 [==============================] - 0s 77us/step - loss: 0.0315 - accuracy: 1.0000 - val_loss: 0.1352 - val_accuracy: 0.9919
Epoch 838/1000
287/287 [==============================] - 0s 95us/step - loss: 0.0315 - accuracy: 1.0000 - val_loss: 0.1351 - val_accuracy: 0.9919
Epoch 839/1000
287/287 [==============================] - 0s 101us/step - loss: 0.0313 - accuracy: 1.0000 - val_loss: 0.1352 - val_accuracy: 0.9919
Epoch 840/1000
287/287 [==============================] - 0s 72us/step - loss: 0.0313 - accuracy: 1.0000 - val_loss: 0.1353 - val_accuracy: 0.9919
Epoch 841/1000
287/287 [==============================] - 0s 83us/step - loss: 0.0313 - accuracy: 1.0000 - val_loss: 0.1353 - val_accuracy: 0.9919
Epoch 842/1000
287/287 [==============================] - 0s 83us/step - loss: 0.0318 - accuracy: 1.0000 - val_loss: 0.1356 - val_accuracy: 0.9919
Epoch 843/1000
287/287 [==============================] - 0s 81us/step - loss: 0.0315 - accuracy: 1.0000 - val_loss: 

Epoch 893/1000
287/287 [==============================] - 0s 87us/step - loss: 0.0292 - accuracy: 1.0000 - val_loss: 0.1365 - val_accuracy: 0.9919
Epoch 894/1000
287/287 [==============================] - 0s 91us/step - loss: 0.0291 - accuracy: 1.0000 - val_loss: 0.1365 - val_accuracy: 0.9919
Epoch 895/1000
287/287 [==============================] - 0s 80us/step - loss: 0.0287 - accuracy: 1.0000 - val_loss: 0.1366 - val_accuracy: 0.9919
Epoch 896/1000
287/287 [==============================] - 0s 78us/step - loss: 0.0290 - accuracy: 1.0000 - val_loss: 0.1366 - val_accuracy: 0.9919
Epoch 897/1000
287/287 [==============================] - 0s 99us/step - loss: 0.0288 - accuracy: 1.0000 - val_loss: 0.1367 - val_accuracy: 0.9919
Epoch 898/1000
287/287 [==============================] - 0s 79us/step - loss: 0.0284 - accuracy: 1.0000 - val_loss: 0.1370 - val_accuracy: 0.9919
Epoch 899/1000
287/287 [==============================] - 0s 107us/step - loss: 0.0286 - accuracy: 1.0000 - val_loss: 

Epoch 949/1000
287/287 [==============================] - 0s 82us/step - loss: 0.0263 - accuracy: 1.0000 - val_loss: 0.1380 - val_accuracy: 0.9919
Epoch 950/1000
287/287 [==============================] - 0s 94us/step - loss: 0.0267 - accuracy: 1.0000 - val_loss: 0.1380 - val_accuracy: 0.9919
Epoch 951/1000
287/287 [==============================] - 0s 77us/step - loss: 0.0266 - accuracy: 1.0000 - val_loss: 0.1382 - val_accuracy: 0.9919
Epoch 952/1000
287/287 [==============================] - 0s 79us/step - loss: 0.0266 - accuracy: 1.0000 - val_loss: 0.1383 - val_accuracy: 0.9919
Epoch 953/1000
287/287 [==============================] - 0s 80us/step - loss: 0.0262 - accuracy: 1.0000 - val_loss: 0.1382 - val_accuracy: 0.9919
Epoch 954/1000
287/287 [==============================] - 0s 82us/step - loss: 0.0263 - accuracy: 1.0000 - val_loss: 0.1385 - val_accuracy: 0.9919
Epoch 955/1000
287/287 [==============================] - 0s 84us/step - loss: 0.0261 - accuracy: 1.0000 - val_loss: 0

In [33]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

124/124 [==============================] - 0s 77us/step
combination test accuracy: 99.19%


In [34]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [35]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [36]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 287 samples, validate on 124 samples
Epoch 1/1000
287/287 [==============================] - 0s 602us/step - loss: 2.0495 - accuracy: 0.5714 - val_loss: 0.8870 - val_accuracy: 0.5161
Epoch 2/1000
287/287 [==============================] - 0s 94us/step - loss: 2.1207 - accuracy: 0.6132 - val_loss: 0.8428 - val_accuracy: 0.5161
Epoch 3/1000
287/287 [==============================] - 0s 129us/step - loss: 2.1601 - accuracy: 0.6655 - val_loss: 0.8096 - val_accuracy: 0.5403
Epoch 4/1000
287/287 [==============================] - 0s 121us/step - loss: 2.1952 - accuracy: 0.6655 - val_loss: 0.7832 - val_accuracy: 0.5565
Epoch 5/1000
287/287 [==============================] - 0s 176us/step - loss: 2.3224 - accuracy: 0.7387 - val_loss: 0.7622 - val_accuracy: 0.5565
Epoch 6/1000
287/287 [==============================] - 0s 136us/step - loss: 1.9078 - accuracy: 0.7561 - val_loss: 0.7454 - val_accuracy: 0.5565
Epoch 7/1000
287/287 [==============================] - 0s 130us/step - loss: 2

287/287 [==============================] - 0s 121us/step - loss: 1.9900 - accuracy: 0.7526 - val_loss: 0.5328 - val_accuracy: 0.8710
Epoch 57/1000
287/287 [==============================] - 0s 126us/step - loss: 2.1692 - accuracy: 0.7282 - val_loss: 0.5327 - val_accuracy: 0.8629
Epoch 58/1000
287/287 [==============================] - 0s 105us/step - loss: 2.3159 - accuracy: 0.7247 - val_loss: 0.5309 - val_accuracy: 0.8629
Epoch 59/1000
287/287 [==============================] - 0s 91us/step - loss: 2.0565 - accuracy: 0.7596 - val_loss: 0.5268 - val_accuracy: 0.8710
Epoch 60/1000
287/287 [==============================] - 0s 88us/step - loss: 1.9394 - accuracy: 0.7631 - val_loss: 0.5253 - val_accuracy: 0.8710
Epoch 61/1000
287/287 [==============================] - 0s 97us/step - loss: 1.8271 - accuracy: 0.7700 - val_loss: 0.5220 - val_accuracy: 0.8710
Epoch 62/1000
287/287 [==============================] - 0s 112us/step - loss: 2.1765 - accuracy: 0.7596 - val_loss: 0.5187 - val_accur

Epoch 112/1000
287/287 [==============================] - 0s 84us/step - loss: 1.7954 - accuracy: 0.8397 - val_loss: 0.4245 - val_accuracy: 0.9113
Epoch 113/1000
287/287 [==============================] - 0s 94us/step - loss: 1.7937 - accuracy: 0.8502 - val_loss: 0.4229 - val_accuracy: 0.9113
Epoch 114/1000
287/287 [==============================] - 0s 89us/step - loss: 2.4722 - accuracy: 0.8084 - val_loss: 0.4219 - val_accuracy: 0.9113
Epoch 115/1000
287/287 [==============================] - 0s 79us/step - loss: 1.2031 - accuracy: 0.8711 - val_loss: 0.4197 - val_accuracy: 0.9113
Epoch 116/1000
287/287 [==============================] - 0s 95us/step - loss: 1.8381 - accuracy: 0.8502 - val_loss: 0.4183 - val_accuracy: 0.9113
Epoch 117/1000
287/287 [==============================] - 0s 86us/step - loss: 1.7122 - accuracy: 0.8606 - val_loss: 0.4158 - val_accuracy: 0.9113
Epoch 118/1000
287/287 [==============================] - 0s 90us/step - loss: 1.9951 - accuracy: 0.8328 - val_loss: 0

Epoch 168/1000
287/287 [==============================] - 0s 97us/step - loss: 1.5586 - accuracy: 0.8815 - val_loss: 0.3539 - val_accuracy: 0.9355
Epoch 169/1000
287/287 [==============================] - 0s 92us/step - loss: 1.9988 - accuracy: 0.8571 - val_loss: 0.3532 - val_accuracy: 0.9355
Epoch 170/1000
287/287 [==============================] - 0s 94us/step - loss: 1.8787 - accuracy: 0.8641 - val_loss: 0.3531 - val_accuracy: 0.9355
Epoch 171/1000
287/287 [==============================] - 0s 85us/step - loss: 1.7833 - accuracy: 0.8746 - val_loss: 0.3515 - val_accuracy: 0.9355
Epoch 172/1000
287/287 [==============================] - 0s 93us/step - loss: 1.8768 - accuracy: 0.8676 - val_loss: 0.3509 - val_accuracy: 0.9355
Epoch 173/1000
287/287 [==============================] - 0s 94us/step - loss: 1.5138 - accuracy: 0.8920 - val_loss: 0.3497 - val_accuracy: 0.9355
Epoch 174/1000
287/287 [==============================] - 0s 96us/step - loss: 1.9881 - accuracy: 0.8641 - val_loss: 0

287/287 [==============================] - 0s 113us/step - loss: 1.8058 - accuracy: 0.8676 - val_loss: 0.3144 - val_accuracy: 0.9435
Epoch 224/1000
287/287 [==============================] - 0s 86us/step - loss: 1.9014 - accuracy: 0.8676 - val_loss: 0.3138 - val_accuracy: 0.9435
Epoch 225/1000
287/287 [==============================] - 0s 113us/step - loss: 1.6421 - accuracy: 0.8780 - val_loss: 0.3134 - val_accuracy: 0.9435
Epoch 226/1000
287/287 [==============================] - 0s 91us/step - loss: 1.2104 - accuracy: 0.9129 - val_loss: 0.3123 - val_accuracy: 0.9435
Epoch 227/1000
287/287 [==============================] - 0s 100us/step - loss: 1.6246 - accuracy: 0.8850 - val_loss: 0.3120 - val_accuracy: 0.9435
Epoch 228/1000
287/287 [==============================] - 0s 89us/step - loss: 1.9564 - accuracy: 0.8606 - val_loss: 0.3119 - val_accuracy: 0.9435
Epoch 229/1000
287/287 [==============================] - 0s 85us/step - loss: 1.5200 - accuracy: 0.8920 - val_loss: 0.3111 - val_

...

Epoch 777/1000
287/287 [==============================] - 0s 99us/step - loss: 1.6312 - accuracy: 0.8990 - val_loss: 0.1994 - val_accuracy: 0.9919
Epoch 778/1000
287/287 [==============================] - 0s 100us/step - loss: 1.3653 - accuracy: 0.9164 - val_loss: 0.2016 - val_accuracy: 0.9919
Epoch 779/1000
287/287 [==============================] - 0s 168us/step - loss: 1.5788 - accuracy: 0.9024 - val_loss: 0.2026 - val_accuracy: 0.9919
Epoch 780/1000
287/287 [==============================] - 0s 189us/step - loss: 1.8445 - accuracy: 0.8850 - val_loss: 0.2000 - val_accuracy: 0.9919
Epoch 781/1000
287/287 [==============================] - 0s 240us/step - loss: 1.8994 - accuracy: 0.8815 - val_loss: 0.2004 - val_accuracy: 0.9919
Epoch 782/1000
287/287 [==============================] - 0s 162us/step - loss: 1.3643 - accuracy: 0.9164 - val_loss: 0.2021 - val_accuracy: 0.9919
Epoch 783/1000
287/287 [==============================] - 0s 102us/step - loss: 1.6832 - accuracy: 0.8955 - val_l

287/287 [==============================] - 0s 77us/step - loss: 1.5215 - accuracy: 0.9059 - val_loss: 0.1979 - val_accuracy: 0.9919
Epoch 833/1000
287/287 [==============================] - 0s 97us/step - loss: 1.6274 - accuracy: 0.8990 - val_loss: 0.1980 - val_accuracy: 0.9919
Epoch 834/1000
287/287 [==============================] - 0s 104us/step - loss: 1.4150 - accuracy: 0.9129 - val_loss: 0.1963 - val_accuracy: 0.9919
Epoch 835/1000
287/287 [==============================] - 0s 90us/step - loss: 1.7370 - accuracy: 0.8920 - val_loss: 0.1975 - val_accuracy: 0.9919
Epoch 836/1000
287/287 [==============================] - 0s 90us/step - loss: 1.7362 - accuracy: 0.8920 - val_loss: 0.1971 - val_accuracy: 0.9919
Epoch 837/1000
287/287 [==============================] - 0s 106us/step - loss: 1.6803 - accuracy: 0.8955 - val_loss: 0.1972 - val_accuracy: 0.9919
Epoch 838/1000
287/287 [==============================] - 0s 154us/step - loss: 1.5742 - accuracy: 0.9024 - val_loss: 0.1992 - val_

287/287 [==============================] - 0s 85us/step - loss: 2.0586 - accuracy: 0.8711 - val_loss: 0.1934 - val_accuracy: 0.9919
Epoch 888/1000
287/287 [==============================] - 0s 83us/step - loss: 1.4102 - accuracy: 0.9129 - val_loss: 0.1937 - val_accuracy: 0.9919
Epoch 889/1000
287/287 [==============================] - 0s 103us/step - loss: 1.7323 - accuracy: 0.8920 - val_loss: 0.1933 - val_accuracy: 0.9919
Epoch 890/1000
287/287 [==============================] - 0s 102us/step - loss: 1.9489 - accuracy: 0.8780 - val_loss: 0.1958 - val_accuracy: 0.9919
Epoch 891/1000
287/287 [==============================] - 0s 90us/step - loss: 1.5701 - accuracy: 0.9024 - val_loss: 0.1933 - val_accuracy: 0.9919
Epoch 892/1000
287/287 [==============================] - 0s 100us/step - loss: 1.9477 - accuracy: 0.8780 - val_loss: 0.1944 - val_accuracy: 0.9919
Epoch 893/1000
287/287 [==============================] - 0s 89us/step - loss: 1.4608 - accuracy: 0.9094 - val_loss: 0.1941 - val_

Epoch 943/1000
287/287 [==============================] - 0s 100us/step - loss: 1.7834 - accuracy: 0.8885 - val_loss: 0.1945 - val_accuracy: 0.9758
Epoch 944/1000
287/287 [==============================] - 0s 90us/step - loss: 1.5160 - accuracy: 0.9059 - val_loss: 0.1946 - val_accuracy: 0.9758
Epoch 945/1000
287/287 [==============================] - 0s 89us/step - loss: 1.8345 - accuracy: 0.8850 - val_loss: 0.1916 - val_accuracy: 0.9919
Epoch 946/1000
287/287 [==============================] - 0s 94us/step - loss: 1.9440 - accuracy: 0.8780 - val_loss: 0.1906 - val_accuracy: 0.9919
Epoch 947/1000
287/287 [==============================] - 0s 88us/step - loss: 1.5167 - accuracy: 0.9059 - val_loss: 0.1912 - val_accuracy: 0.9919
Epoch 948/1000
287/287 [==============================] - 0s 104us/step - loss: 1.8392 - accuracy: 0.8850 - val_loss: 0.1931 - val_accuracy: 0.9758
Epoch 949/1000
287/287 [==============================] - 0s 86us/step - loss: 1.5693 - accuracy: 0.9024 - val_loss:

287/287 [==============================] - 0s 83us/step - loss: 1.6218 - accuracy: 0.8990 - val_loss: 0.1893 - val_accuracy: 0.9919
Epoch 999/1000
287/287 [==============================] - 0s 96us/step - loss: 1.3523 - accuracy: 0.9164 - val_loss: 0.1887 - val_accuracy: 0.9919
Epoch 1000/1000
287/287 [==============================] - 0s 93us/step - loss: 2.0527 - accuracy: 0.8711 - val_loss: 0.1902 - val_accuracy: 0.9758


In [37]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

124/124 [==============================] - 0s 82us/step
combination test accuracy: 97.58%


In [38]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [39]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [40]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 335us/step - loss: 0.8270 - accuracy: 0.4592 - val_loss: 0.6718 - val_accuracy: 0.6713
Epoch 2/1000
331/331 [==============================] - 0s 84us/step - loss: 0.6856 - accuracy: 0.6737 - val_loss: 0.6274 - val_accuracy: 0.6224
Epoch 3/1000
331/331 [==============================] - 0s 104us/step - loss: 0.6496 - accuracy: 0.6344 - val_loss: 0.6051 - val_accuracy: 0.7063
Epoch 4/1000
331/331 [==============================] - 0s 104us/step - loss: 0.6263 - accuracy: 0.6677 - val_loss: 0.5875 - val_accuracy: 0.7063
Epoch 5/1000
331/331 [==============================] - 0s 92us/step - loss: 0.6055 - accuracy: 0.6616 - val_loss: 0.5737 - val_accuracy: 0.7063
Epoch 6/1000
331/331 [==============================] - 0s 93us/step - loss: 0.5910 - accuracy: 0.6647 - val_loss: 0.5634 - val_accuracy: 0.7063
Epoch 7/1000
331/331 [==============================] - 0s 98us/step - loss: 0.57

331/331 [==============================] - 0s 128us/step - loss: 0.3706 - accuracy: 0.8943 - val_loss: 0.3939 - val_accuracy: 0.8252
Epoch 57/1000
331/331 [==============================] - 0s 153us/step - loss: 0.3674 - accuracy: 0.8792 - val_loss: 0.3886 - val_accuracy: 0.8252
Epoch 58/1000
331/331 [==============================] - 0s 88us/step - loss: 0.3646 - accuracy: 0.8912 - val_loss: 0.3850 - val_accuracy: 0.8322
Epoch 59/1000
331/331 [==============================] - 0s 80us/step - loss: 0.3635 - accuracy: 0.8973 - val_loss: 0.3835 - val_accuracy: 0.8252
Epoch 60/1000
331/331 [==============================] - 0s 78us/step - loss: 0.3596 - accuracy: 0.8912 - val_loss: 0.3835 - val_accuracy: 0.8252
Epoch 61/1000
331/331 [==============================] - 0s 85us/step - loss: 0.3582 - accuracy: 0.8943 - val_loss: 0.3827 - val_accuracy: 0.8252
Epoch 62/1000
331/331 [==============================] - 0s 83us/step - loss: 0.3550 - accuracy: 0.8852 - val_loss: 0.3769 - val_accurac

331/331 [==============================] - 0s 79us/step - loss: 0.2618 - accuracy: 0.9305 - val_loss: 0.3033 - val_accuracy: 0.8601
Epoch 113/1000
331/331 [==============================] - 0s 83us/step - loss: 0.2619 - accuracy: 0.9275 - val_loss: 0.3012 - val_accuracy: 0.8601
Epoch 114/1000
331/331 [==============================] - 0s 82us/step - loss: 0.2599 - accuracy: 0.9305 - val_loss: 0.2940 - val_accuracy: 0.8741
Epoch 115/1000
331/331 [==============================] - 0s 76us/step - loss: 0.2579 - accuracy: 0.9305 - val_loss: 0.2895 - val_accuracy: 0.8811
Epoch 116/1000
331/331 [==============================] - 0s 74us/step - loss: 0.2564 - accuracy: 0.9366 - val_loss: 0.2880 - val_accuracy: 0.8811
Epoch 117/1000
331/331 [==============================] - 0s 85us/step - loss: 0.2579 - accuracy: 0.9366 - val_loss: 0.2870 - val_accuracy: 0.8811
Epoch 118/1000
331/331 [==============================] - 0s 81us/step - loss: 0.2534 - accuracy: 0.9396 - val_loss: 0.2918 - val_acc

Epoch 168/1000
331/331 [==============================] - 0s 81us/step - loss: 0.1978 - accuracy: 0.9456 - val_loss: 0.2401 - val_accuracy: 0.8881
Epoch 169/1000
331/331 [==============================] - 0s 80us/step - loss: 0.1970 - accuracy: 0.9456 - val_loss: 0.2399 - val_accuracy: 0.8881
Epoch 170/1000
331/331 [==============================] - 0s 82us/step - loss: 0.1967 - accuracy: 0.9456 - val_loss: 0.2446 - val_accuracy: 0.8811
Epoch 171/1000
331/331 [==============================] - 0s 77us/step - loss: 0.1964 - accuracy: 0.9396 - val_loss: 0.2371 - val_accuracy: 0.8881
Epoch 172/1000
331/331 [==============================] - 0s 79us/step - loss: 0.1950 - accuracy: 0.9486 - val_loss: 0.2397 - val_accuracy: 0.8881
Epoch 173/1000
331/331 [==============================] - 0s 81us/step - loss: 0.1930 - accuracy: 0.9486 - val_loss: 0.2424 - val_accuracy: 0.8811
Epoch 174/1000
331/331 [==============================] - 0s 77us/step - loss: 0.1929 - accuracy: 0.9456 - val_loss: 0

331/331 [==============================] - 0s 81us/step - loss: 0.1593 - accuracy: 0.9607 - val_loss: 0.2097 - val_accuracy: 0.9161
Epoch 224/1000
331/331 [==============================] - 0s 78us/step - loss: 0.1577 - accuracy: 0.9668 - val_loss: 0.2130 - val_accuracy: 0.9161
Epoch 225/1000
331/331 [==============================] - 0s 87us/step - loss: 0.1570 - accuracy: 0.9607 - val_loss: 0.2115 - val_accuracy: 0.9161
Epoch 226/1000
331/331 [==============================] - 0s 81us/step - loss: 0.1573 - accuracy: 0.9637 - val_loss: 0.2134 - val_accuracy: 0.9161
Epoch 227/1000
331/331 [==============================] - 0s 80us/step - loss: 0.1551 - accuracy: 0.9607 - val_loss: 0.2018 - val_accuracy: 0.9231
Epoch 228/1000
331/331 [==============================] - 0s 81us/step - loss: 0.1569 - accuracy: 0.9547 - val_loss: 0.2029 - val_accuracy: 0.9301
Epoch 229/1000
331/331 [==============================] - 0s 95us/step - loss: 0.1575 - accuracy: 0.9637 - val_loss: 0.2024 - val_acc

...

331/331 [==============================] - 0s 80us/step - loss: 0.0449 - accuracy: 0.9970 - val_loss: 0.1199 - val_accuracy: 0.9790
Epoch 725/1000
331/331 [==============================] - 0s 89us/step - loss: 0.0451 - accuracy: 0.9970 - val_loss: 0.1257 - val_accuracy: 0.9720
Epoch 726/1000
331/331 [==============================] - 0s 88us/step - loss: 0.0448 - accuracy: 0.9970 - val_loss: 0.1249 - val_accuracy: 0.9720
Epoch 727/1000
331/331 [==============================] - 0s 75us/step - loss: 0.0444 - accuracy: 0.9970 - val_loss: 0.1209 - val_accuracy: 0.9720
Epoch 728/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0442 - accuracy: 0.9970 - val_loss: 0.1225 - val_accuracy: 0.9720
Epoch 729/1000
331/331 [==============================] - 0s 83us/step - loss: 0.0441 - accuracy: 0.9970 - val_loss: 0.1267 - val_accuracy: 0.9720
Epoch 730/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0442 - accuracy: 0.9970 - val_loss: 0.1258 - val_acc

Epoch 780/1000
331/331 [==============================] - 0s 91us/step - loss: 0.0402 - accuracy: 0.9970 - val_loss: 0.1257 - val_accuracy: 0.9720
Epoch 781/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0402 - accuracy: 0.9970 - val_loss: 0.1226 - val_accuracy: 0.9720
Epoch 782/1000
331/331 [==============================] - 0s 79us/step - loss: 0.0403 - accuracy: 0.9970 - val_loss: 0.1231 - val_accuracy: 0.9720
Epoch 783/1000
331/331 [==============================] - 0s 82us/step - loss: 0.0402 - accuracy: 0.9970 - val_loss: 0.1258 - val_accuracy: 0.9720
Epoch 784/1000
331/331 [==============================] - 0s 78us/step - loss: 0.0401 - accuracy: 0.9970 - val_loss: 0.1270 - val_accuracy: 0.9720
Epoch 785/1000
331/331 [==============================] - 0s 90us/step - loss: 0.0399 - accuracy: 0.9940 - val_loss: 0.1226 - val_accuracy: 0.9720
Epoch 786/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0401 - accuracy: 0.9970 - val_loss: 0

Epoch 836/1000
331/331 [==============================] - 0s 85us/step - loss: 0.0366 - accuracy: 0.9970 - val_loss: 0.1212 - val_accuracy: 0.9720
Epoch 837/1000
331/331 [==============================] - 0s 96us/step - loss: 0.0366 - accuracy: 0.9970 - val_loss: 0.1213 - val_accuracy: 0.9720
Epoch 838/1000
331/331 [==============================] - 0s 85us/step - loss: 0.0365 - accuracy: 0.9970 - val_loss: 0.1186 - val_accuracy: 0.9720
Epoch 839/1000
331/331 [==============================] - 0s 79us/step - loss: 0.0362 - accuracy: 0.9970 - val_loss: 0.1262 - val_accuracy: 0.9720
Epoch 840/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0368 - accuracy: 0.9970 - val_loss: 0.1226 - val_accuracy: 0.9720
Epoch 841/1000
331/331 [==============================] - 0s 82us/step - loss: 0.0363 - accuracy: 0.9970 - val_loss: 0.1157 - val_accuracy: 0.9720
Epoch 842/1000
331/331 [==============================] - 0s 84us/step - loss: 0.0365 - accuracy: 0.9970 - val_loss: 0

Epoch 892/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0339 - accuracy: 0.9970 - val_loss: 0.1230 - val_accuracy: 0.9720
Epoch 893/1000
331/331 [==============================] - 0s 85us/step - loss: 0.0336 - accuracy: 0.9970 - val_loss: 0.1210 - val_accuracy: 0.9720
Epoch 894/1000
331/331 [==============================] - 0s 82us/step - loss: 0.0336 - accuracy: 0.9970 - val_loss: 0.1221 - val_accuracy: 0.9720
Epoch 895/1000
331/331 [==============================] - 0s 77us/step - loss: 0.0337 - accuracy: 0.9970 - val_loss: 0.1208 - val_accuracy: 0.9720
Epoch 896/1000
331/331 [==============================] - 0s 79us/step - loss: 0.0338 - accuracy: 0.9970 - val_loss: 0.1198 - val_accuracy: 0.9720
Epoch 897/1000
331/331 [==============================] - 0s 86us/step - loss: 0.0337 - accuracy: 0.9970 - val_loss: 0.1188 - val_accuracy: 0.9720
Epoch 898/1000
331/331 [==============================] - 0s 85us/step - loss: 0.0332 - accuracy: 0.9970 - val_loss: 0

Epoch 948/1000
331/331 [==============================] - 0s 86us/step - loss: 0.0309 - accuracy: 0.9970 - val_loss: 0.1192 - val_accuracy: 0.9720
Epoch 949/1000
331/331 [==============================] - 0s 81us/step - loss: 0.0309 - accuracy: 0.9970 - val_loss: 0.1198 - val_accuracy: 0.9720
Epoch 950/1000
331/331 [==============================] - 0s 73us/step - loss: 0.0308 - accuracy: 0.9970 - val_loss: 0.1194 - val_accuracy: 0.9720
Epoch 951/1000
331/331 [==============================] - 0s 81us/step - loss: 0.0309 - accuracy: 0.9970 - val_loss: 0.1184 - val_accuracy: 0.9720
Epoch 952/1000
331/331 [==============================] - 0s 76us/step - loss: 0.0307 - accuracy: 0.9970 - val_loss: 0.1192 - val_accuracy: 0.9720
Epoch 953/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0308 - accuracy: 0.9970 - val_loss: 0.1211 - val_accuracy: 0.9720
Epoch 954/1000
331/331 [==============================] - 0s 88us/step - loss: 0.0307 - accuracy: 0.9970 - val_loss: 0

In [41]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 111us/step
over-sampling test accuracy: 97.90%


In [42]:
#### add regularizer and dropout on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [43]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 599us/step - loss: 2.5591 - accuracy: 0.6888 - val_loss: 0.9327 - val_accuracy: 0.6154
Epoch 2/1000
331/331 [==============================] - 0s 120us/step - loss: 2.6803 - accuracy: 0.6435 - val_loss: 0.8743 - val_accuracy: 0.6084
Epoch 3/1000
331/331 [==============================] - 0s 106us/step - loss: 2.2721 - accuracy: 0.6616 - val_loss: 0.8193 - val_accuracy: 0.6503
Epoch 4/1000
331/331 [==============================] - 0s 98us/step - loss: 2.1406 - accuracy: 0.6707 - val_loss: 0.7813 - val_accuracy: 0.6503
Epoch 5/1000
331/331 [==============================] - 0s 109us/step - loss: 1.7915 - accuracy: 0.6979 - val_loss: 0.7440 - val_accuracy: 0.6364
Epoch 6/1000
331/331 [==============================] - 0s 104us/step - loss: 2.3523 - accuracy: 0.6707 - val_loss: 0.7227 - val_accuracy: 0.6503
Epoch 7/1000
331/331 [==============================] - 0s 104us/step - loss: 2

331/331 [==============================] - 0s 133us/step - loss: 1.7040 - accuracy: 0.8338 - val_loss: 0.5229 - val_accuracy: 0.8601
Epoch 57/1000
331/331 [==============================] - 0s 135us/step - loss: 1.9720 - accuracy: 0.8097 - val_loss: 0.5243 - val_accuracy: 0.8322
Epoch 58/1000
331/331 [==============================] - 0s 112us/step - loss: 1.7299 - accuracy: 0.8338 - val_loss: 0.5195 - val_accuracy: 0.8601
Epoch 59/1000
331/331 [==============================] - 0s 97us/step - loss: 1.5058 - accuracy: 0.8338 - val_loss: 0.5171 - val_accuracy: 0.8671
Epoch 60/1000
331/331 [==============================] - 0s 102us/step - loss: 1.9550 - accuracy: 0.8218 - val_loss: 0.5154 - val_accuracy: 0.8392
Epoch 61/1000
331/331 [==============================] - 0s 130us/step - loss: 1.8226 - accuracy: 0.8278 - val_loss: 0.5138 - val_accuracy: 0.8671
Epoch 62/1000
331/331 [==============================] - 0s 110us/step - loss: 1.7685 - accuracy: 0.8218 - val_loss: 0.5109 - val_acc

Epoch 112/1000
331/331 [==============================] - 0s 98us/step - loss: 1.5054 - accuracy: 0.8459 - val_loss: 0.4377 - val_accuracy: 0.9091
Epoch 113/1000
331/331 [==============================] - 0s 99us/step - loss: 1.5070 - accuracy: 0.8429 - val_loss: 0.4348 - val_accuracy: 0.9091
Epoch 114/1000
331/331 [==============================] - 0s 107us/step - loss: 1.6820 - accuracy: 0.8459 - val_loss: 0.4314 - val_accuracy: 0.8881
Epoch 115/1000
331/331 [==============================] - 0s 133us/step - loss: 1.6913 - accuracy: 0.8459 - val_loss: 0.4304 - val_accuracy: 0.8881
Epoch 116/1000
331/331 [==============================] - 0s 166us/step - loss: 1.6898 - accuracy: 0.8459 - val_loss: 0.4333 - val_accuracy: 0.9091
Epoch 117/1000
331/331 [==============================] - 0s 248us/step - loss: 1.5908 - accuracy: 0.8489 - val_loss: 0.4295 - val_accuracy: 0.9091
Epoch 118/1000
331/331 [==============================] - 0s 143us/step - loss: 1.7683 - accuracy: 0.8369 - val_lo

331/331 [==============================] - 0s 87us/step - loss: 1.8162 - accuracy: 0.8399 - val_loss: 0.3794 - val_accuracy: 0.9161
Epoch 168/1000
331/331 [==============================] - 0s 91us/step - loss: 1.9024 - accuracy: 0.8369 - val_loss: 0.3794 - val_accuracy: 0.9371
Epoch 169/1000
331/331 [==============================] - 0s 86us/step - loss: 1.6632 - accuracy: 0.8640 - val_loss: 0.3757 - val_accuracy: 0.9091
Epoch 170/1000
331/331 [==============================] - 0s 77us/step - loss: 1.7701 - accuracy: 0.8399 - val_loss: 0.3781 - val_accuracy: 0.9371
Epoch 171/1000
331/331 [==============================] - 0s 84us/step - loss: 1.7248 - accuracy: 0.8489 - val_loss: 0.3764 - val_accuracy: 0.8951
Epoch 172/1000
331/331 [==============================] - 0s 85us/step - loss: 1.2996 - accuracy: 0.8882 - val_loss: 0.3899 - val_accuracy: 0.9161
Epoch 173/1000
331/331 [==============================] - 0s 80us/step - loss: 1.8093 - accuracy: 0.8580 - val_loss: 0.3783 - val_acc

331/331 [==============================] - 0s 105us/step - loss: 1.9219 - accuracy: 0.8550 - val_loss: 0.3407 - val_accuracy: 0.9301
Epoch 223/1000
331/331 [==============================] - 0s 118us/step - loss: 2.0624 - accuracy: 0.8429 - val_loss: 0.3406 - val_accuracy: 0.9301
Epoch 224/1000
331/331 [==============================] - 0s 106us/step - loss: 1.8282 - accuracy: 0.8610 - val_loss: 0.3417 - val_accuracy: 0.9301
Epoch 225/1000
331/331 [==============================] - 0s 97us/step - loss: 1.3628 - accuracy: 0.8912 - val_loss: 0.3380 - val_accuracy: 0.9091
Epoch 226/1000
331/331 [==============================] - 0s 97us/step - loss: 1.8580 - accuracy: 0.8701 - val_loss: 0.3382 - val_accuracy: 0.9301
Epoch 227/1000
331/331 [==============================] - 0s 113us/step - loss: 1.5381 - accuracy: 0.8761 - val_loss: 0.3375 - val_accuracy: 0.9301
Epoch 228/1000
331/331 [==============================] - 0s 120us/step - loss: 1.5926 - accuracy: 0.8761 - val_loss: 0.3371 - va

...

Epoch 724/1000
331/331 [==============================] - 0s 95us/step - loss: 1.5827 - accuracy: 0.8943 - val_loss: 0.2261 - val_accuracy: 0.9720
Epoch 725/1000
331/331 [==============================] - 0s 89us/step - loss: 1.8665 - accuracy: 0.8761 - val_loss: 0.2248 - val_accuracy: 0.9720
Epoch 726/1000
331/331 [==============================] - 0s 86us/step - loss: 1.5373 - accuracy: 0.9003 - val_loss: 0.2247 - val_accuracy: 0.9720
Epoch 727/1000
331/331 [==============================] - 0s 81us/step - loss: 1.4833 - accuracy: 0.9094 - val_loss: 0.2249 - val_accuracy: 0.9720
Epoch 728/1000
331/331 [==============================] - 0s 85us/step - loss: 1.6274 - accuracy: 0.8912 - val_loss: 0.2251 - val_accuracy: 0.9720
Epoch 729/1000
331/331 [==============================] - 0s 88us/step - loss: 2.0970 - accuracy: 0.8580 - val_loss: 0.2246 - val_accuracy: 0.9720
Epoch 730/1000
331/331 [==============================] - 0s 86us/step - loss: 1.9077 - accuracy: 0.8761 - val_loss: 0

Epoch 780/1000
331/331 [==============================] - 0s 94us/step - loss: 1.6286 - accuracy: 0.8912 - val_loss: 0.2216 - val_accuracy: 0.9790
Epoch 781/1000
331/331 [==============================] - 0s 93us/step - loss: 1.8132 - accuracy: 0.8792 - val_loss: 0.2211 - val_accuracy: 0.9720
Epoch 782/1000
331/331 [==============================] - 0s 87us/step - loss: 1.7201 - accuracy: 0.8852 - val_loss: 0.2236 - val_accuracy: 0.9790
Epoch 783/1000
331/331 [==============================] - 0s 89us/step - loss: 2.2291 - accuracy: 0.8550 - val_loss: 0.2213 - val_accuracy: 0.9720
Epoch 784/1000
331/331 [==============================] - 0s 81us/step - loss: 1.6746 - accuracy: 0.8882 - val_loss: 0.2215 - val_accuracy: 0.9720
Epoch 785/1000
331/331 [==============================] - 0s 85us/step - loss: 1.5343 - accuracy: 0.8943 - val_loss: 0.2215 - val_accuracy: 0.9790
Epoch 786/1000
331/331 [==============================] - 0s 98us/step - loss: 1.4866 - accuracy: 0.9003 - val_loss: 0

331/331 [==============================] - 0s 95us/step - loss: 1.8068 - accuracy: 0.8822 - val_loss: 0.2171 - val_accuracy: 0.9790
Epoch 836/1000
331/331 [==============================] - 0s 97us/step - loss: 1.8497 - accuracy: 0.8822 - val_loss: 0.2188 - val_accuracy: 0.9720
Epoch 837/1000
331/331 [==============================] - 0s 95us/step - loss: 1.7202 - accuracy: 0.8852 - val_loss: 0.2177 - val_accuracy: 0.9790
Epoch 838/1000
331/331 [==============================] - 0s 94us/step - loss: 1.8129 - accuracy: 0.8792 - val_loss: 0.2172 - val_accuracy: 0.9790
Epoch 839/1000
331/331 [==============================] - 0s 90us/step - loss: 1.5216 - accuracy: 0.9033 - val_loss: 0.2174 - val_accuracy: 0.9790
Epoch 840/1000
331/331 [==============================] - 0s 110us/step - loss: 1.4793 - accuracy: 0.9063 - val_loss: 0.2173 - val_accuracy: 0.9790
Epoch 841/1000
331/331 [==============================] - 0s 93us/step - loss: 1.2972 - accuracy: 0.9124 - val_loss: 0.2257 - val_ac

Epoch 891/1000
331/331 [==============================] - 0s 96us/step - loss: 1.5708 - accuracy: 0.8973 - val_loss: 0.2141 - val_accuracy: 0.9790
Epoch 892/1000
331/331 [==============================] - 0s 103us/step - loss: 1.2494 - accuracy: 0.9154 - val_loss: 0.2150 - val_accuracy: 0.9790
Epoch 893/1000
331/331 [==============================] - 0s 87us/step - loss: 1.8556 - accuracy: 0.8761 - val_loss: 0.2141 - val_accuracy: 0.9790
Epoch 894/1000
331/331 [==============================] - 0s 96us/step - loss: 1.9900 - accuracy: 0.8701 - val_loss: 0.2140 - val_accuracy: 0.9790
Epoch 895/1000
331/331 [==============================] - 0s 90us/step - loss: 1.8968 - accuracy: 0.8761 - val_loss: 0.2193 - val_accuracy: 0.9790
Epoch 896/1000
331/331 [==============================] - 0s 91us/step - loss: 1.9951 - accuracy: 0.8671 - val_loss: 0.2143 - val_accuracy: 0.9790
Epoch 897/1000
331/331 [==============================] - 0s 94us/step - loss: 1.5715 - accuracy: 0.8973 - val_loss: 

Epoch 947/1000
331/331 [==============================] - 0s 110us/step - loss: 1.6628 - accuracy: 0.8943 - val_loss: 0.2116 - val_accuracy: 0.9790
Epoch 948/1000
331/331 [==============================] - 0s 85us/step - loss: 1.5686 - accuracy: 0.8973 - val_loss: 0.2114 - val_accuracy: 0.9790
Epoch 949/1000
331/331 [==============================] - 0s 102us/step - loss: 1.9436 - accuracy: 0.8731 - val_loss: 0.2117 - val_accuracy: 0.9790
Epoch 950/1000
331/331 [==============================] - 0s 92us/step - loss: 1.6212 - accuracy: 0.8943 - val_loss: 0.2115 - val_accuracy: 0.9790
Epoch 951/1000
331/331 [==============================] - 0s 91us/step - loss: 1.7605 - accuracy: 0.8822 - val_loss: 0.2121 - val_accuracy: 0.9790
Epoch 952/1000
331/331 [==============================] - 0s 95us/step - loss: 1.7066 - accuracy: 0.8912 - val_loss: 0.2109 - val_accuracy: 0.9790
Epoch 953/1000
331/331 [==============================] - 0s 109us/step - loss: 1.5202 - accuracy: 0.9033 - val_loss

In [45]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 108us/step
over-sampling test accuracy: 97.90%


In [46]:
############ Logistic Regression ############

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [48]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 97.58%


In [50]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 95.10%


In [52]:
############## Random Forest ##############

In [53]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [54]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.19%


In [55]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [56]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 95.80%


In [57]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [58]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.98275862 1.         0.94827586 1.         0.98214286]
0.9826354679802956


In [59]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.98507463 0.96969697 0.98484848 0.98484848 0.93939394]
0.97277250113071
